In [77]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_yaml


In [78]:
# Read in the data (copy cel from song-train).
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('abba.csv')

# Concatenate all lyrics into a single string
corpus = ' '.join(data['lyrics'].dropna())

# Print the first 500 characters of the corpus just to verify
print(corpus[:500])

My my
At Waterloo Napoleon did surrender
Oh yeah
And I have met my destiny in quite a similar way
The history book on the shelf
Is always repeating itself
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
My my
I tried to hold you back, but you were stronger
Oh yeah
And now it seems my only chance is giving up the fight
And how co


In [79]:
# Clean the data. (copy cell from song-train)
import re

# Function to clean the text
def clean_text(text):
    # Remove Unicode characters
    cleaned_text = re.sub('[^\x00-\x7F]+', '', text)
    # Remove \r and other special characters
    cleaned_text = re.sub(r'[\r\n\t]', ' ', cleaned_text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Clean the corpus
corpus = clean_text(corpus)

# Save the cleaned corpus to a file
with open('cleaned_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(corpus)

print("Data cleaned and saved to 'cleaned_corpus.txt'.")

Data cleaned and saved to 'cleaned_corpus.txt'.


In [80]:
# Create encoder and decoder dictionaries. (copy-cel from song-train)
# Get unique characters from the corpus
unique_chars = sorted(set(corpus))

# Create encoder dictionary (character to integer)
encoder_dict = {char: i for i, char in enumerate(unique_chars)}

# Create decoder dictionary (integer to character)
decoder_dict = {i: char for i, char in enumerate(unique_chars)}

# Print encoder dictionary
print("Encoder dictionary:")
print(encoder_dict)

# Print decoder dictionary
print("\nDecoder dictionary:")
print(decoder_dict)

Encoder dictionary:
{' ': 0, '!': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '6': 15, ':': 16, '?': 17, 'A': 18, 'B': 19, 'C': 20, 'D': 21, 'E': 22, 'F': 23, 'G': 24, 'H': 25, 'I': 26, 'J': 27, 'K': 28, 'L': 29, 'M': 30, 'N': 31, 'O': 32, 'P': 33, 'Q': 34, 'R': 35, 'S': 36, 'T': 37, 'U': 38, 'V': 39, 'W': 40, 'Y': 41, 'Z': 42, '[': 43, ']': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, '\x7f': 71}

Decoder dictionary:
{0: ' ', 1: '!', 2: '"', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '/', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '6', 16: ':', 17: '?', 18: 'A', 19: 'B', 20: 'C', 21: 'D', 22: 'E', 23: 'F', 24: 'G', 25: 'H', 26: 'I', 27: 'J', 28: 'K', 29: 'L', 30: 'M', 31: 'N', 32: 'O', 33: 'P', 34: 'Q', 35: 'R', 36

In [81]:
# Read model from json file and load trained weights from hdf5 file.
# UITZOEKEN VIA DE TENSOFLOW SITE HOE DIT MOET.
# b.v.  https://machinelearningmastery.com/save-load-keras-deep-learning-models/
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
for i in range(19):
    model.load_weights('./weights-' + str(i + 1).zfill(2) + '.hdf5')

In [82]:
# Uitleg in de les

def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [83]:
# Uitleg in de les
num_chars = len(unique_chars)
corpus_length = len(corpus)

def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=bool)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoder_dict[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            pred = model.predict(X, verbose=0)[0]
            prediction = sample(pred, 0.3)

            generated_text += decoder_dict[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

sentence_length = 20
def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]

        return pattern

In [84]:

seed = make_seed('In the bard and show you on your lovelight and i can\'t get the mowner i\'m a marion an and every mind, there\'s a boot')
print(seed, end=" ")
txt =  generate(seed)
print(txt)

In the bard and show  I can do I want that you're a light I see that you can the time is the same is all be and the pay We're think to say and say and the sare of of a mide I for the same of the dark on the mime I was a man And the same it still say the bean I can do and the seest of the and I was to the same of me with no more me we have to be with you I me I was that I have to say a bean I was strough the darking on the more The same of the midnight the supar I was the time is all the sare I was to the same it was


## Sample song #1
In the bard and show you on your lovelight and i can't get the mowner i'm a marion an and every mind, there's a boot

## Sample song #1
in the bard and show i don't be a world be the song i mave the was the so the street you the the be me

## Sample song 3 
in the bard and show a stay it the fart the the me the the the all i ding you the the now the song the the the so some